# Neural Machine Translation: German to English

Implementing an encoder-decoder neural machine translator (NMT) with single Long-Short Term Memory (LSTM) layer using standard TensorFlow operations.

In [1]:

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os

# imported request module to get translation text data from url
import requests
import io
import random
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve

from time import time
# Dual Nvida GEFORCE RTX 2080 8GB GPU
# tensorflow-gpu 1.11
# cuDNN v7.4.2.24 for Windows 10
# CUDA Toolkit v9.0 for Windows 10
# MS Visual Studio 2017 Complimentary
# Python 3.6.6 (64-bit)
import tensorflow as tf
from PIL import Image
from collections import Counter
import matplotlib.gridspec as gridspec
# import word2vec
from gensim.models import Word2Vec
import nltk
from nltk.translate.bleu_score import corpus_bleu


## Loading Data 

First, download the data from this [page](https://nlp.stanford.edu/projects/nmt/). The required files are:

* File containing German sentences: [`train.de`](https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de)
* File containing English sentences: [`train.en`](https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en)


### Loading Training and Testing Data

Here we load the data and split the data in the files into two sets; training and testing data.

In [2]:
data_dir = 'temp'
train_de_file = 'train.de'
train_en_file = 'train.en'

print('Loading German Training Data')

# Download German vocab text data if does not exist
if not os.path.isfile(os.path.join(data_dir, train_de_file)):
    print ('Data not Found, downloading German Training Data from URL')
    train_de_url = 'https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de'
    r = requests.get(train_de_url)
    # file_de_train = r.read('train.de')
    file_de_train = r.content
    # Format Data
    train_de_data = file_de_train.decode()
    train_de_data = train_de_data.encode('ascii',errors='ignore')
    # Decode to text string format  
    # Make string into list based on newline space 
    train_de_data = train_de_data.decode().split('\n')
    # Write sentence list to file
    with open(os.path.join(data_dir, train_de_file), 'w') as out_conn:
        for sent_tokens in train_de_data:
            # write german sentence tokens to file
            out_conn.write(sent_tokens + '\n')

# Contains the training sentences
source_sent = [] # Input
target_sent = [] # Output

# Contains the testing sentences
test_source_sent = [] # Input
test_target_sent = [] # Output

# We grab around 100 lines of data that are interleaved 
# in the first 50000 sentences
test_indices = [l_i for l_i in range(50,50001,500)]

# Read the source data file and read the first 250,000 lines (except first 50)
with open(os.path.join(data_dir, train_de_file), encoding='utf-8') as f:
    for l_i, line in enumerate(f):
        # discarding first 50 translations as there was some
        # english to english mappings found in the first few lines. which are wrong
        if l_i<50:
            continue
        
        if len(source_sent)<250000 and l_i not in test_indices:
            source_sent.append(line)
        elif l_i in test_indices:
            test_source_sent.append(line)

print('Loading English Training Data')

# Download German vocab text data if does not exist
if not os.path.isfile(os.path.join(data_dir, train_en_file)):
    print ('Data not Found, downloading English Training Data from URL')
    train_en_url = 'https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en'
    r = requests.get(train_en_url)
    # file_en_train = r.read('train.en')
    file_en_train = r.content
    # Format Data
    train_en_data = file_en_train.decode()
    train_en_data = train_en_data.encode('ascii',errors='ignore')
    # Decode to text string format  
    # Make string into list based on newline space 
    train_en_data = train_en_data.decode().split('\n')
    # Write sentence list to file
    with open(os.path.join(data_dir, train_en_file), 'w') as out_conn:
        for sent_tokens in train_en_data:
            # write english sentence tokens to file
            out_conn.write(sent_tokens + '\n')
            
# Read the target data file and read the first 250,000 lines (except first 50)            
with open(os.path.join(data_dir, train_en_file), encoding='utf-8') as f:
    for l_i, line in enumerate(f):
        # discarding first 50 translations as there was some
        # english to english mappings found in the first few lines. which are wrong
        if l_i<50:
            continue
        
        if len(target_sent)<250000 and l_i not in test_indices:
            target_sent.append(line)
        elif l_i in test_indices:
            test_target_sent.append(line)
        
# Make sure we extracted same number of both extracted source and target sentences         
assert len(source_sent)==len(target_sent),'Source: %d, Target: %d'%(len(source_sent),len(target_sent))

# Print some source sentences
print('Sample translations (%d)'%len(source_sent))
for i in range(0,250000,10000):
    print('(',i,') DE: ', source_sent[i])
    print('(',i,') EN: ', target_sent[i])

# Print some target sentences
print('Sample test translations (%d)'%len(test_source_sent))
for i in range(0,100,10):
    print('DE: ', test_source_sent[i])
    print('EN: ', test_target_sent[i])



Loading German Training Data
Loading English Training Data
Sample translations (250000)
( 0 ) DE:  Hier erfahren Sie , wie Sie Creative Suite 2 und Creative Suite 3 am besten zusammen mit QuarkXPress nutzen knnen .

( 0 ) EN:  Here , you  ll find out how Creative Suite users can get the best possible interaction with QuarkXPress .

( 10000 ) DE:  Fr die sehr gnstigen Wochen- und Monatskarten ( 1 Monat ca.

( 10000 ) EN:  It is THE trendy area of Marseille .

( 20000 ) DE:  Freuen Sie sich auf die romantische Atmosphre in den Zimmern und Apartments .

( 20000 ) EN:  Enjoy the romantic atmosphere of one of the guest rooms or apartments .

( 30000 ) DE:  Zu zwiespltig sind Dr. Gutherzens Erfahrungen aus frhen Studententagen verlaufen , in denen er sich in die Gefielde von durch Heidegger geprgten Autor / innen begeben hat und dort stndig mit strengem Blick darauf verwiesen wurde , er habe bestimmte Theorieressourcen und Gedankengebude einfach noch nicht grndlich genug verstanden und knne 

In [3]:
# Preprocess source and target training datasets for embedding modeling
source_sent_list = []
for sent in source_sent:
    # Remove punctuation and new-line chars
    sent = sent.replace(',',' ')
    sent = sent.replace('.',' ')
    sent = sent.replace('\n',' ')
    sent_list = sent.split(' ')
    # Insert start token at the beginning of the sentence list
    sent_list.insert(0,'<s>')
    # Add end token at the end of the sentence list
    sent_list.append('</s>')
    source_sent_list.append(sent_list)
    
print('Source sentence list shape is',len(source_sent_list))
print(source_sent_list[0])
print()
    
target_sent_list = []
for sent in target_sent:
    # Remove punctuation and new-line chars
    sent = sent.replace(',',' ')
    sent = sent.replace('.',' ')
    sent = sent.replace('\n',' ')
    sent_list = sent.split(' ')
    sent_list.insert(0,'<s>')
    sent_list.append('</s>')
    target_sent_list.append(sent_list)
    
print('Target sentence list shape is',len(target_sent_list))
print(target_sent_list[0])

Source sentence list shape is 250000
['<s>', 'Hier', 'erfahren', 'Sie', '', '', 'wie', 'Sie', 'Creative', 'Suite', '2', 'und', 'Creative', 'Suite', '3', 'am', 'besten', 'zusammen', 'mit', 'QuarkXPress', 'nutzen', 'knnen', '', '', '', '</s>']

Target sentence list shape is 250000
['<s>', 'Here', '', '', 'you', '', 'll', 'find', 'out', 'how', 'Creative', 'Suite', 'users', 'can', 'get', 'the', 'best', 'possible', 'interaction', 'with', 'QuarkXPress', '', '', '', '</s>']


In [4]:
# Use source and target dataset for training token embedding model and building vocabularies

embedding_size = 128 # Dimension of the embedding vector.

# Generate German token embeddings with gensim Word2Vec
model_de = Word2Vec(source_sent_list, size=embedding_size, min_count=6)
model_de_token_list = list(model_de.wv.vocab)
print('German token vocab embedding number is:', len(model_de_token_list))

# source tokens with embeddings mapped to index 
src_dictionary = dict()

# the loop to generate token2index dictionary
for i, src_vocab_token in enumerate(model_de_token_list):
    src_dictionary[src_vocab_token] = i

# Build a reverse dictionary with the model mapping index to token (swap keys and values)
src_reverse_dictionary = dict(zip(src_dictionary.values(),src_dictionary.keys()))
      

# Print some of the words in the dictionary
print('Source')
print('\t',list(src_dictionary.items())[(len(src_dictionary)-10):len(src_dictionary)])
print('\t',list(src_reverse_dictionary.items())[:10])
print('\t','Vocabulary size: ', len(src_dictionary))
print()

model_de.save('/nmt_embeddings/de_embeddings')

# Generate English token embeddings with gensim Word2Vec
model_en = Word2Vec(target_sent_list, size=embedding_size, min_count=4)
model_en_token_list = list(model_en.wv.vocab)
print('English token vocab embedding number is:', len(model_en_token_list))

# target tokens with embeddings mapped to index
tgt_dictionary = dict()

# the loop to generate token2index dictionary
for i, tgt_vocab_token in enumerate(model_en_token_list):
      tgt_dictionary[tgt_vocab_token] = i

# Build a reverse dictionary with the model mapping index to token (swap keys and values)
tgt_reverse_dictionary = dict(zip(tgt_dictionary.values(),tgt_dictionary.keys()))
      

model_en.save('/nmt_embeddings/en_embeddings')

# Print some of the words in the dictionary
print('Target')
print('\t',list(tgt_dictionary.items())[:10])
print('\t',list(tgt_reverse_dictionary.items())[:10])
print('\t','Vocabulary size: ', len(tgt_dictionary))

# Each language has 50000 words
vocabulary_size_src = len(src_dictionary)
vocabulary_size_tgt = len(tgt_dictionary)

German token vocab embedding number is: 45903
Source
	 [('Pereira', 45893), ('Mitochondrien', 45894), ('CURLAUTH', 45895), ('Sasse', 45896), ('ingres', 45897), ('zh', 45898), ('Drexler', 45899), ('Finnegan', 45900), ('Volunteers', 45901), ('BODY', 45902)]
	 [(0, '<s>'), (1, 'Hier'), (2, 'erfahren'), (3, 'Sie'), (4, ''), (5, 'wie'), (6, 'Creative'), (7, 'Suite'), (8, '2'), (9, 'und')]
	 Vocabulary size:  45903

English token vocab embedding number is: 45569
Target
	 [('<s>', 0), ('Here', 1), ('', 2), ('you', 3), ('ll', 4), ('find', 5), ('out', 6), ('how', 7), ('Creative', 8), ('Suite', 9)]
	 [(0, '<s>'), (1, 'Here'), (2, ''), (3, 'you'), (4, 'll'), (5, 'find'), (6, 'out'), (7, 'how'), (8, 'Creative'), (9, 'Suite')]
	 Vocabulary size:  45569


### Preprocessing text
Here we preprocess the text by replacing words not found in the dictionary with `<unk>` as well as remove punctuation marks (`.`,`,`) and new-line characters.

In [5]:
# Keep track of how many unknown words were encountered
src_unk_count, tgt_unk_count = 0, 0

def split_to_tokens(sent,is_source):
    '''
    This function takes in a sentence (source or target)
    and preprocess the sentences with various steps (e.g. removing punctuation)
    '''
    
    global src_unk_count, tgt_unk_count

    # Remove punctuation and new-line chars
    sent = sent.replace(',',' ')
    sent = sent.replace('.',' ')
    sent = sent.replace('\n',' ') 
    
    sent_toks = sent.split(' ')
    for t_i, tok in enumerate(sent_toks):
        if is_source:
            # src_dictionary contain the word -> word ID mapping for source vocabulary
            if tok not in src_dictionary.keys():
                if not len(tok.strip())==0:
                    sent_toks[t_i] = '<unk>'
                    src_unk_count += 1
        else:
            # tgt_dictionary contain the word -> word ID mapping for target vocabulary
            if tok not in tgt_dictionary.keys():
                if not len(tok.strip())==0:
                    sent_toks[t_i] = '<unk>'
                    #print(tok)
                    tgt_unk_count += 1
    return sent_toks


# Let us first look at some statistics of the sentences
# Train - source data
source_len = []
source_mean, source_std = 0,0
for sent in source_sent:
    temp_split_sent_list = split_to_tokens(sent,True)
    source_len.append(len(temp_split_sent_list))
    

print('(Source) Sentence mean length: ', np.mean(source_len))
print('(Source) Sentence stddev length: ', np.std(source_len))
print()


# Let us first look at some statistics of the sentences
# Train - target data
target_len = []

for sent in target_sent:
    temp_split_sent_list = split_to_tokens(sent,False)
    target_len.append(len(temp_split_sent_list))
    

print('(Target) Sentence mean length: ', np.mean(target_len))
print('(Target) Sentence stddev length: ', np.std(target_len))
print()

    
  
# Let us first look at some statistics of the sentences
# Test - source data
test_source_len = []
for sent in test_source_sent:
    test_source_len.append(len(split_to_tokens(sent, True)))
    
print('(Test-Source) Sentence mean length: ', np.mean(test_source_len))
print('(Test-Source) Sentence stddev length: ', np.std(test_source_len))

# Let us first look at some statistics of the sentences
# Test - target data
test_target_len = []
test_tgt_mean, test_tgt_std = 0,0
for sent in test_target_sent:
    test_target_len.append(len(split_to_tokens(sent, False)))
    
print('(Test-Target) Sentence mean length: ', np.mean(test_target_len))
print('(Test-Target) Sentence stddev length: ', np.std(test_target_len))



(Source) Sentence mean length:  26.244692
(Source) Sentence stddev length:  13.854376414156501

(Target) Sentence mean length:  28.275308
(Target) Sentence stddev length:  14.925498769057468

(Test-Source) Sentence mean length:  26.61
(Test-Source) Sentence stddev length:  14.800604717375572
(Test-Target) Sentence mean length:  29.08
(Test-Target) Sentence stddev length:  16.19424589167399


### Making training and testing data fixed length

Here we get all the source sentences and target sentences to a fixed length. This is, so that we can process the sentences as batches.

In [6]:
# ================================================================================
# Processing training data

src_unk_count, tgt_unk_count = 0, 0

train_inputs = []
train_outputs = []

# Chosen based on previously found statistics
src_max_sent_length = 41 
tgt_max_sent_length = 61

print('Processing Training Data ...\n')
for s_i, (src_sent, tgt_sent) in enumerate(zip(source_sent,target_sent)):
    # Break source and target sentences to word lists
    src_sent_tokens = split_to_tokens(src_sent,True)
    tgt_sent_tokens = split_to_tokens(tgt_sent,False)
    
    # Append <s> token's ID to the beginning of source sentence
    num_src_sent = [src_dictionary['<s>']]
    # Add the rest of word IDs for words found in the source sentence 
    for tok in src_sent_tokens:
        if tok in src_dictionary.keys():
            num_src_sent.append(src_dictionary[tok])

    # If the length of the source sentence below the maximum allowed length
    # append </s> token's ID to the end
    if len(num_src_sent)<src_max_sent_length:
        num_src_sent.extend([src_dictionary['</s>'] for _ in range(src_max_sent_length - len(num_src_sent))])

    # If the length exceed the maximum allowed length
    # truncate the sentence
    elif len(num_src_sent)>src_max_sent_length:
        num_src_sent = num_src_sent[:src_max_sent_length]
        
    # Make sure the sentence is of length src_max_sent_length
    assert len(num_src_sent)==src_max_sent_length,len(num_src_sent)

    train_inputs.append(num_src_sent)
    
    # Create the numeric target sentence with word IDs
    # append <s> to the beginning and append actual words later
    num_tgt_sent = [tgt_dictionary['<s>']]
    for tok in tgt_sent_tokens:
        if tok in tgt_dictionary.keys():
            num_tgt_sent.append(tgt_dictionary[tok])
        
    ## Modifying the outputs such that all the outputs have max_length elements
    if len(num_tgt_sent)<tgt_max_sent_length:
        num_tgt_sent.extend([tgt_dictionary['</s>'] for _ in range(tgt_max_sent_length - len(num_tgt_sent))])
    elif len(num_tgt_sent)>tgt_max_sent_length:
        num_tgt_sent = num_tgt_sent[:tgt_max_sent_length]
        
    train_outputs.append(num_tgt_sent)
    
print('Unk counts Src: %d, Tgt: %d'%(src_unk_count, tgt_unk_count))
print('Sentences ',len(train_inputs))

assert len(train_inputs)  == len(source_sent),\
        'Size of total elements: %d, Total sentences: %d'\
                %(len(train_inputs),len(source_sent))

# Making inputs and outputs NumPy arrays
train_inputs = np.array(train_inputs, dtype=np.int32)
train_outputs = np.array(train_outputs, dtype=np.int32)

# Make sure number of inputs and outputs dividable by 100
train_inputs = train_inputs[:(train_inputs.shape[0]//100)*100,:]
train_outputs = train_outputs[:(train_outputs.shape[0]//100)*100,:]
print('\t Done processing training data \n')

# Printing some data
print('Samples from training data')
for ti in range(10):
    print('\t',[src_reverse_dictionary[w]  for w in train_inputs[ti,:].tolist()])
    print('\t',[tgt_reverse_dictionary[w]  for w in train_outputs[ti,:].tolist()])
print()
print('\tSentences ',train_inputs.shape[0])

# ================================================================================
# Processing Test data

src_unk_count, tgt_unk_count = 0, 0
print('Processing testing data ....\n')
test_inputs = []
test_outputs = []
for s_i, (src_sent,tgt_sent) in enumerate(zip(test_source_sent,test_target_sent)):
    src_sent_tokens = split_to_tokens(src_sent,True)
    tgt_sent_tokens = split_to_tokens(tgt_sent,False)
    
    num_src_sent = [src_dictionary['<s>']]
    for tok in src_sent_tokens:
        if tok in src_dictionary.keys():
            num_src_sent.append(src_dictionary[tok])
    
    num_tgt_sent = [src_dictionary['<s>']]
    for tok in tgt_sent_tokens:
        if tok in tgt_dictionary.keys():
            num_tgt_sent.append(tgt_dictionary[tok])
        
    # Append </s> if the length is not src_max_sent_length
    if len(num_src_sent)<src_max_sent_length:
        num_src_sent.extend([src_dictionary['</s>'] for _ in range(src_max_sent_length - len(num_src_sent))])
    # Truncate the sentence if length is over src_max_sent_length
    elif len(num_src_sent)>src_max_sent_length:
        num_src_sent = num_src_sent[:src_max_sent_length]
        
    assert len(num_src_sent)==src_max_sent_length, len(num_src_sent)

    test_inputs.append(num_src_sent)
    
    # Append </s> is length is not tgt_max_sent_length
    if len(num_tgt_sent)<tgt_max_sent_length:
        num_tgt_sent.extend([tgt_dictionary['</s>'] for _ in range(tgt_max_sent_length - len(num_tgt_sent))])
    # Truncate the sentence if length over tgt_max_sent_length
    elif len(num_tgt_sent)>tgt_max_sent_length:
        num_tgt_sent = num_tgt_sent[:tgt_max_sent_length]
        
    assert len(num_tgt_sent)==tgt_max_sent_length, len(num_tgt_sent)

    test_outputs.append(num_tgt_sent)

# Printing some data
print('Unk counts Tgt: %d, Tgt: %d'%(src_unk_count, tgt_unk_count))    
print('Done processing testing data ....\n')
test_inputs = np.array(test_inputs,dtype=np.int32)
test_outputs = np.array(test_outputs,dtype=np.int32)
print('Samples from training data')
for ti in range(10):
    print('\t',[src_reverse_dictionary[w]  for w in test_inputs[ti,:].tolist()])
    print('\t',[tgt_reverse_dictionary[w]  for w in test_outputs[ti,:].tolist()])

Processing Training Data ...

Unk counts Src: 350002, Tgt: 142756
Sentences  250000
	 Done processing training data 

Samples from training data
	 ['<s>', 'Hier', 'erfahren', 'Sie', '', '', 'wie', 'Sie', 'Creative', 'Suite', '2', 'und', 'Creative', 'Suite', '3', 'am', 'besten', 'zusammen', 'mit', 'QuarkXPress', 'nutzen', 'knnen', '', '', '', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
	 ['<s>', 'Here', '', '', 'you', '', 'll', 'find', 'out', 'how', 'Creative', 'Suite', 'users', 'can', 'get', 'the', 'best', 'possible', 'interaction', 'with', 'QuarkXPress', '', '', '', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
	 ['<s>', 'Sie', 'werden', 'berrascht', 'sein'

In [7]:


# Total number of sentences
tot_sentences = train_inputs.shape[0]
print('Total number of training sentences: ',tot_sentences)

# we keep a cursor for each sentence in the training set
sentence_cursors = [0 for _ in range(tot_sentences)] 

batch_size = 64


Total number of training sentences:  250000


## Flipping the Input Data
Changin the order of the sentence of the target language improves the performance of NMT systems. Because when reversed, it helps the NMT system to establish a strong connection as the last word of the source language and the last word of the target language will be closest to each other. *DON'T RUN THIS MULTIPLE TIMES as running two times gives original.*

In [8]:
## Reverse the German sentences
# Remember reversing the source sentence gives better performance
# DON'T RUN THIS MULTIPLE TIMES as running two times gives original
train_inputs = np.fliplr(train_inputs)
test_inputs = np.fliplr(test_inputs)

print('Training and Test source data after flipping ')
print('\t',[src_reverse_dictionary[w] for w in train_inputs[0,:].tolist()])
print('\t',[tgt_reverse_dictionary[w] for w in test_inputs[0,:].tolist()])
print()
print('\t',[src_reverse_dictionary[w] for w in train_inputs[10,:].tolist()])
print('\t',[tgt_reverse_dictionary[w] for w in test_inputs[10,:].tolist()])

print()
print('\nTesting data after flipping')
print('\t',[src_reverse_dictionary[w] for w in test_inputs[0,:].tolist()])

Training and Test source data after flipping 
	 ['</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '', '', '', 'knnen', 'nutzen', 'QuarkXPress', 'mit', 'zusammen', 'besten', 'am', '3', 'Suite', 'Creative', 'und', '2', 'Suite', 'Creative', 'Sie', 'wie', '', '', 'Sie', 'erfahren', 'Hier', '<s>']
	 ['up', 'Quality', 'many', 'mentions', 'has', 'Suite', 'll', 'll', 'gTr', 'detected', '&amp;', 'peaceful', 'possible', 'many', 'drives', 'with', 'll', 'directly', 'Suite', 'opened', 'brings', 'll', 'll', 'To', 'Airconditioning', 'HTML', 'depths', 'll', 'including', 'Suite', 'll', 'any', 'll', 'll', 'by', 'll', 'possible', 'easy', 'AND', 'Sites', '<s>']

	 ['</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '', '', '', ')', 'Import', '##AT##-##AT##', 'PSD', '&gt;', 'Fenster', '(', 'Import', '##AT##-##AT##', 'PSD', 'Palette', 'die', 'Sie', 'ffnen', '', '', 'knnen', 'zu', 'nutzen', 'Dateien', 

## Data Generations for MT

Now we define the data generator for our NMT.

In [9]:
input_size = embedding_size

class DataGeneratorMT(object):
    
    def __init__(self,batch_size,num_unroll,is_source, is_train):
        # Number of data points in a batch
        self._batch_size = batch_size
        # Number of unrollings
        self._num_unroll = num_unroll
        # Cursors for each element in batch
        self._cursor = [0 for offset in range(self._batch_size)]
        
        
        # Define trained word embeddings models
        self._src_word_embeddings = model_de
        self._tgt_word_embeddings = model_en
        
        # The sentence IDs being currently processed to create the 
        # current batch
        self._sent_ids = None
        
        # We want a batch of data from source or target?
        self._is_source = is_source
        # Is this training or testing data?
        self._is_train = is_train
                
    def next_batch(self, sent_ids):
        
        # Depending on whether we want source or target data
        # change the maximum sentence length
        if self._is_source:
            max_sent_length = src_max_sent_length
            vocabulary_size = vocabulary_size_src
        else:
            max_sent_length = tgt_max_sent_length
            vocabulary_size = vocabulary_size_tgt
            
            
        # Arrays to hold input and output data
        # Word embeddings (current word)
        batch_data = np.zeros((self._batch_size,input_size),dtype=np.float32)
        # One-hot encoded label (next word)
        batch_labels = np.zeros((self._batch_size,vocabulary_size),dtype=np.float32)
        
        
        # Populate each index of the batch
        for b in range(self._batch_size):
            
            # Sentence IDs to get data from
            sent_id = sent_ids[b]
            
            # If generating data with source sentences
            # use src_word_embeddings
            if self._is_source:
                # Depending on whether we need training data or test data
                # choose the previously created training or test data
                if self._is_train:
                    sent_text = train_inputs[sent_id]
                else:
                    sent_text = test_inputs[sent_id]
                             
                # Populate the batch data arrays
                # Generate batch word embeddings
                batch_data[b] = np.array(self._src_word_embeddings[src_reverse_dictionary[sent_text[self._cursor[b]]]])
                batch_labels[b] = np.zeros((vocabulary_size),dtype=np.float32)
                # One-hot encode output labels to predict the next word token  
                batch_labels[b,sent_text[self._cursor[b]+1]] = 1.0
                
           
                
            # If generating data with target sentences
            # use tgt_word_embeddings
            else:
                # Depending on whether we need training data or testind data
                # choose the previously created training or testind data
                if self._is_train:
                    sent_text = train_outputs[sent_id]
                else:
                    sent_text = test_outputs[sent_id]
                
                # We cannot avoid having two different embedding vectors for <s> token
                # in soruce and target languages
                # Therefore, if the symbol appears, we always take the source embedding vector
                if sent_text[self._cursor[b]]!=tgt_dictionary['<s>']:
                    batch_data[b] = np.array(self._tgt_word_embeddings[tgt_reverse_dictionary[sent_text[self._cursor[b]]]])
                else:
                    batch_data[b] = np.array(self._src_word_embeddings[src_reverse_dictionary[sent_text[self._cursor[b]]]])
                
                # Populate the data arrays
                batch_labels[b] = np.zeros((vocabulary_size),dtype=np.float32)
                # One-hot encode output labels to predict the next word token
                batch_labels[b,sent_text[self._cursor[b]+1]] = 1.0
                                  
            
            # Update the cursor for each batch index
            self._cursor[b] = (self._cursor[b]+1)%(max_sent_length-1)
             
        return batch_data,batch_labels
        
    def unroll_batches(self,sent_ids):
        
        # Only if new sentence IDs if provided
        # else it will use the previously defined 
        # sent_ids continuously
        if sent_ids is not None:
            
            self._sent_ids = sent_ids
            # Unlike in the previous exercises we do not process a single sequence
            # over many iterations of unrollings. We process either a source sentence or target sentence
            # at a single go. So we reset the _cursor evrytime we generate a batch
            self._cursor = [0 for _ in range(self._batch_size)]
                
        unroll_data,unroll_labels = [],[]
        
        # Unrolling data over time
        for ui in range(self._num_unroll):
            
            if self._is_source:
                data, labels = self.next_batch(self._sent_ids)
            else:
                data, labels = self.next_batch(self._sent_ids)
                    
            unroll_data.append(data)
            unroll_labels.append(labels)
        
        # Return unrolled data and sentence IDs
        return unroll_data, unroll_labels, self._sent_ids
    
    def reset_indices(self):
        self._cursor = [0 for offset in range(self._batch_size)]
        
# Running a tiny set to see if the implementation correct
dg = DataGeneratorMT(batch_size=5,num_unroll=20,is_source=True, is_train=True)
u_data, u_labels, _ = dg.unroll_batches([0,1,2,3,4])

for _, lbl in zip(u_data,u_labels):
    unrolled_line_list = []
    # print('Unrolled label lines shape',lbl.shape)
    for i in range(lbl.shape[0]):
        unrolled_line_list.append(src_reverse_dictionary[np.argmax(lbl[i])])
    print(unrolled_line_list)

dg = DataGeneratorMT(batch_size=5,num_unroll=30,is_source=False, is_train=True)
u_data, u_labels, _ = dg.unroll_batches([0,2,3,4,5])
print('\nTarget data batch')
for _,lbl in zip(u_data,u_labels):
    unrolled_line_list = []
    # print('Unrolled label lines shape',lbl.shape)
    for i in range(lbl.shape[0]):
        unrolled_line_list.append(tgt_reverse_dictionary[np.argmax(lbl[i])])
    print(unrolled_line_list)

['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '</s>', '</s>', '</s>']
['</s>', '</s>', '', '</s>', '</s>']
['</s>', '</s>', '', '', '</s>']
['</s>', '</s>', '', '', '</s>']
['</s>', '</s>', 'bietet', '', '</s>']
['</s>', '</s>', 'Dateiformat', 'nutzen', '</s>']
['</s>', '</s>', '##AT##-##AT##', 'optimal', '']
['</s>', '</s>', 'PSD', 'Bilder', '']
['', '</s>', 'das', 'Ihre', '']
['', '', 'ber', 'fr', 'werden']
['', '', 'Photoshop', 'es', 'ausgewhlt']
['knnen', '', 'mit', 'Sie', 'Verwendungszweck']
['nutzen', 'lsst', 'Integration', 'wie', 'nach']

Target data batch
['Here', 'QuarkXPress', 'In', 'For', 'If']
['', '8', 'this', 'example', 'you']
['', 'is', 'section', '', 'use']
['you', 'considered', 'we', '', 'PSD']
['', 'by', '', 'you',

c:\users\trevo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
c:\users\trevo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
c:\users\trevo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:85: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


## Defining the NMT with TensorFlow
Now let us define various operations parameters hyperparameters needed to define our NMT model. First we define the hyperparameters, then input/output placeholders, the LSTM/Output layer parameters, LSTM/output computations, and finally optimization steps.

### Defining hyperparameters
Here we define various hyperparameters we use to define our model.

In [10]:
# Initialize numpy array for target (English) embedding matrix
tgt_emb_mat = np.zeros((len(tgt_dictionary)+1,embedding_size),dtype=np.float32)

# Generate vocab list from target (English) embedding model
model_en_token_list = list(model_en.wv.vocab)
print('English Word2Vec Model token list length',len(model_en_token_list))

for token in model_en_token_list:
    # Put token embedding at token dictionary index location in numpy array
    tgt_emb_mat[tgt_dictionary[token]] = model_en[token]

print('Display sample of target embedding matrix')
for i in range(0,tgt_emb_mat.shape[0],(tgt_emb_mat.shape[0]//5)):
    target_token = tgt_reverse_dictionary[i]
    print('Token:', target_token)
    print('\tTarget Token Embedding from matrix:',tgt_emb_mat[i])
   

#tgt_emb_mat = np.load('en-embeddings.npy')
input_size = tgt_emb_mat.shape[1]

num_nodes = 128
batch_size = 10

# We unroll the full length at one go
# both source and target sentences
enc_num_unrollings = 40
dec_num_unrollings = 60


English Word2Vec Model token list length 45569
Display sample of target embedding matrix
Token: <s>
	Target Token Embedding from matrix: [ 0.47202292 -0.19785295 -0.42661235  1.5680743  -0.42968413 -0.91455865
 -1.7425988  -2.3957255   1.5689483   0.72408015  2.0869095   0.07355866
  0.09983094 -0.65684104 -0.396206    0.56921345  1.104658   -1.8643866
  1.6053082  -1.4258722   0.31625572 -0.3445102   0.9164614  -0.0312829
  0.7316988   0.32894948 -0.04728745 -0.4798339  -0.71572113  0.63946706
 -0.79966044  0.41423875  0.09139918  1.6376328  -0.669813    0.77255857
  0.5261138  -0.26791182 -2.5562515  -0.44032004 -0.6725734   2.2908607
 -2.06485     0.7731549  -1.2632537   0.3139722  -1.9448675  -0.04442921
  0.38827026  3.4436023   0.5917488  -0.84818023 -1.8798739  -0.32243907
  1.6038985   0.6957146  -0.33058843  2.178744    0.38926575 -0.45274922
 -0.94427395  0.2341337  -1.0168778  -0.80002815 -0.24367109  1.3378296
 -0.4035711   1.6942079  -2.4811242  -0.10054661 -0.28287527  1.

c:\users\trevo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [11]:
# Initialize numpy array for source (German) embedding matrix
src_emb_mat = np.zeros((len(src_dictionary)+1,embedding_size),dtype=np.float32)

# Generate vocab list from target (English) embedding model
model_de_token_list = list(model_de.wv.vocab)
print('German Word2Vec Model token list length',len(model_de_token_list))

for token in model_de_token_list:
    # Put token embedding at token dictionary index location in numpy array
    src_emb_mat[src_dictionary[token]] = model_de[token]

print('Display sample of source embedding matrix')
for i in range(0,src_emb_mat.shape[0],(src_emb_mat.shape[0]//5)):
    target_token = src_reverse_dictionary[i]
    print('Token:', target_token)
    print('\tSource Token Embedding from matrix:',src_emb_mat[i])
    # print('\tToken Embedding from model:',model_en[target_token])
    print()

German Word2Vec Model token list length 45903
Display sample of source embedding matrix
Token: <s>
	Source Token Embedding from matrix: [ 2.07613850e+00  4.53027278e-01  6.64695799e-01  1.36244678e+00
 -8.04737210e-02 -2.01550961e+00 -5.94697654e-01  7.78674185e-01
  1.56722879e+00 -2.61353421e+00 -9.56043154e-02 -1.08640872e-01
 -9.79179680e-01  7.22788632e-01 -1.57551062e+00 -6.88399136e-01
  1.34107172e+00 -7.13573322e-02 -1.67309809e+00 -4.96853888e-01
 -1.03360879e+00  2.77371198e-01  1.87715769e+00 -5.10850012e-01
  2.31329083e+00 -6.98499799e-01  1.92507672e+00 -4.44678009e-01
  1.26312470e+00  1.60253990e+00  8.70870590e-01  2.19968605e+00
 -6.63858533e-01 -5.20596504e-01  6.65355325e-02  4.34014559e-01
 -9.39516425e-01 -1.20300090e+00 -3.80590200e-01  6.41823173e-01
 -5.86635530e-01 -1.98774922e+00 -7.84082890e-01  6.58075035e-01
  5.08868814e-01  1.46549296e+00 -1.55810273e+00 -1.05815172e+00
  1.06726253e+00  2.49360371e+00  4.07215416e-01 -2.13893831e-01
 -2.45939875e+00  9

c:\users\trevo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


### Defining Input/Output Placeholders
Here we define the placeholder to feed in inputs/outputs. Additionally we define a mask placeholder that can mask certain outputs from the loss calculation.

In [12]:
tf.reset_default_graph()

with tf.device("/device:GPU:1"):
    # Target embeddings are needed to compute the test outputs
    tgt_word_embeddings = tf.convert_to_tensor(tgt_emb_mat,name='tgt_embeddings')

    print('Defining encoder data placeholders')

    # Training Input placeholders (Encoder)
    # Encoder related input data, we directly feed in the embeddings
    enc_train_inputs = []

    # Defining unrolled training inputs
    for ui in range(enc_num_unrollings):
        enc_train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,input_size],name='train_inputs_%d'%ui))

    print('Defining decoder data placeholders')
    # Training Input/Output/Mask data (Decoder)
    # Decoder inputs and outputs
    dec_train_inputs, dec_train_labels = [],[]
    # We use masking to mask out the any of the </s> elements
    # from the loss computation in the decoder
    dec_train_masks = []

    # Defining unrolled training inputs
    for ui in range(dec_num_unrollings):
        dec_train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,input_size],name='dec_train_inputs_%d'%ui))
        dec_train_labels.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size_tgt], name = 'dec_train_labels_%d'%ui))
        dec_train_masks.append(tf.placeholder(tf.float32, shape=[batch_size,1],name='dec_train_masks_%d'%ui))


    enc_test_input = [tf.placeholder(tf.float32, shape=[batch_size,input_size], name='test_input_%d'%ui)\
                      for ui in range(enc_num_unrollings)]
    dec_test_input = tf.nn.embedding_lookup(tgt_word_embeddings,[tgt_dictionary['<s>']])
print('\tDone')

Defining encoder data placeholders
Defining decoder data placeholders
	Done


### Defining the Encoder Model

We define the encoder model. The encoder model is a single LSTM cell with TensorFlow variables for the state and output variables.

In [13]:
print('Defining Encoder Parameters')
with tf.device("/device:GPU:1"):
    with tf.variable_scope('Encoder'):

        # Input gate (i_t) - How much memory to write to cell state
        # We use xavier initialization as this gives better results by reduce vanishing gradients 
        enc_ix = tf.get_variable('ix',shape=[input_size, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        enc_im = tf.get_variable('im',shape=[num_nodes, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        enc_ib = tf.Variable(tf.random_uniform([1, num_nodes],-0.05, 0.05),name='ib')

        # Forget gate (f_t) - How much memory to discard from cell state
        enc_fx = tf.get_variable('fx',shape=[input_size, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        enc_fm = tf.get_variable('fm',shape=[num_nodes, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        enc_fb = tf.Variable(tf.random_uniform([1, num_nodes],-0.05, 0.05),name='fb')

        # Candidate value (c~_t) - Used to compute the current cell state                            
        enc_cx = tf.get_variable('cx',shape=[input_size, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        enc_cm = tf.get_variable('cm',shape=[num_nodes, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        enc_cb = tf.Variable(tf.random_uniform([1, num_nodes],-0.05,0.05),name='cb') 

        # Output gate - How much memory to output from the cell state
        enc_ox = tf.get_variable('ox',shape=[input_size, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        enc_om = tf.get_variable('om',shape=[num_nodes, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        enc_ob = tf.Variable(tf.random_uniform([1, num_nodes],-0.05,0.05),name='ob') 

        # Variables saving state across unrollings (testing).
        saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name='train_output')
        saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False, name = 'train_cell')

        saved_test_output = tf.Variable(tf.zeros([batch_size, num_nodes]),trainable=False, name='test_output')
        saved_test_state = tf.Variable(tf.zeros([batch_size, num_nodes]),trainable=False, name='test_cell')

print('\tDone')

Defining Encoder Parameters
	Done


### Defining the Decoder Model

Decoder is a single LSTM cell with an additional softmax layer that can predict words.

In [14]:
print('Defining Decoder Parameters')

with tf.device("/device:GPU:1"):  
    with tf.variable_scope('Decoder'):

        # Input gate (i_t) - How much memory to write to cell state
        # We use xavier initialization as this gives better results by reducing vanishing gradients
        dec_ix = tf.get_variable('ix',shape=[input_size, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        dec_im = tf.get_variable('im',shape=[num_nodes, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        dec_ib = tf.Variable(tf.random_uniform([1, num_nodes],-0.05, 0.05),name='ib')    

        # Forget gate (f_t) - How much memory to discard from cell state
        dec_fx = tf.get_variable('fx',shape=[input_size, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        dec_fm = tf.get_variable('fm',shape=[num_nodes, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        dec_fb = tf.Variable(tf.random_uniform([1, num_nodes],-0.05, 0.05),name='fb')    

        # Candidate value (c~_t) - Used to compute the current cell state                             
        dec_cx = tf.get_variable('cx',shape=[input_size, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        dec_cm = tf.get_variable('cm',shape=[num_nodes, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        dec_cb = tf.Variable(tf.random_uniform([1, num_nodes],-0.05,0.05),name='cb')     

        # Output gate - How much memory to output from the cell state
        dec_ox = tf.get_variable('ox',shape=[input_size, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        dec_om = tf.get_variable('om',shape=[num_nodes, num_nodes],
                                 initializer = tf.contrib.layers.xavier_initializer())
        dec_ob = tf.Variable(tf.random_uniform([1, num_nodes],-0.05,0.05),name='ob') 

        # Softmax Classifier weights and biases.
        # If we are using sampled softmax loss, the weights dims shouldbe [50000, 64]
        # If not, then [64, 50000]
        w = tf.get_variable('softmax_weights',shape=[num_nodes, vocabulary_size_tgt], 
                            initializer = tf.contrib.layers.xavier_initializer())
        b = tf.Variable(tf.random_uniform([vocabulary_size_tgt],-0.05,-0.05),name='softmax_bias')

print('\tDone')

Defining Decoder Parameters
	Done


### Defining LSTM Computations

Here we first define two function `enc_lstm_cell` and `dec_lstm_cell` which define the LSTM cell computations we discussed in early chapters. Next we define the computations to compute the final state variables of the encoder, feeding that into the decoder as the intial state and finally computing the LSTM output, logit values and the predictions.

In [15]:
# Definition of the cell computation (Encoder)
def enc_lstm_cell(i, o, state):
    """Create a LSTM cell"""
    with tf.device("/device:GPU:1"):
        input_gate = tf.sigmoid(tf.matmul(i, enc_ix) + tf.matmul(o, enc_im) + enc_ib)
        forget_gate = tf.sigmoid(tf.matmul(i, enc_fx) + tf.matmul(o, enc_fm) + enc_fb)
        update = tf.matmul(i, enc_cx) + tf.matmul(o, enc_cm) + enc_cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, enc_ox) + tf.matmul(o, enc_om) + enc_ob)
    return output_gate * tf.tanh(state), state

# Definition of the cell computation (Decoder)
def dec_lstm_cell(i, o, state):
    """Create a LSTM cell"""
    with tf.device("/device:GPU:1"):
        input_gate = tf.sigmoid(tf.matmul(i, dec_ix) + tf.matmul(o, dec_im) + dec_ib)
        forget_gate = tf.sigmoid(tf.matmul(i, dec_fx) + tf.matmul(o, dec_fm) + dec_fb)
        update = tf.matmul(i, dec_cx) + tf.matmul(o, dec_cm) + dec_cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, dec_ox) + tf.matmul(o, dec_om) + dec_ob)
    return output_gate * tf.tanh(state), state

with tf.device("/device:GPU:1"):
    # ================================================
    # Training inference logic
    # Holds the outputs of the all unrolled LSTM steps
    outputs = list()

    # Initialize the output and state variables
    # with zeros and iteratively update these two 
    # variables with the LSTM's output and state
    output = saved_output
    state = saved_state

    print('Calculating Encoder Output')
    # Update the output and state of the encoder iteratively
    for i in enc_train_inputs:
        output, state = enc_lstm_cell(i, output,state)


    print('Calculating Decoder Output')
    # With the computations of the enc_lstm_cell done,
    # calculate the output and state of the decoder
    with tf.control_dependencies([saved_output.assign(output),
                                 saved_state.assign(state)]):
        # Calculate the decoder state and output iteratively
        for i in dec_train_inputs:
            output, state = dec_lstm_cell(i, output, state)
            outputs.append(output)

    # Calculate the logits of the decoder for all unrolled steps
    logits = tf.matmul(tf.concat(axis=0, values=outputs), w) + b

    # Predictions from the decoder
    train_prediction = tf.nn.softmax(logits)

    # ===================================================
    # Testing related inference logic
    # We calculate the test predictions for the maximum 
    # num_unrollings allowed for target language
    test_output  = saved_test_output
    test_state = saved_test_state

    test_predictions = []

    # Compute the encoder output iteratively
    for i in enc_test_input:
        test_output, test_state = enc_lstm_cell(i, test_output,test_state)

    # Compute the decoder output iteratively
    # where the input is whatever the previously output word's embedding
    with tf.control_dependencies([saved_test_output.assign(test_output),
                                     saved_test_state.assign(test_state)]):
        for i in range(dec_num_unrollings):

            test_output, test_state = dec_lstm_cell(dec_test_input, test_output, test_state)

            test_prediction = tf.nn.softmax(tf.nn.xw_plus_b(test_output, w, b))

            dec_test_input = tf.nn.embedding_lookup(tgt_word_embeddings,tf.argmax(test_prediction,axis=1))
            test_predictions.append(tf.argmax(test_prediction,axis=1))

print('\tDone')

Calculating Encoder Output
Calculating Decoder Output
	Done


### Calculating the Loss

Here we calculate the loss. Loss is calculated by summing all the losses obtained across the time axis and averaging over the batch axis.

In [16]:
print('Calculating Loss')

with tf.device("/device:GPU:1"):
    # No need for a tf.control_dependencies(...) clause here
    # Because we restart the state anyway after each sentence batch
    loss_batch = tf.concat(axis=0,values=\
                           dec_train_masks)*tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,\
                                                                                       labels=tf.concat(axis=0,\
                                                                                                        values=dec_train_labels))
    loss = tf.reduce_mean(loss_batch)

Calculating Loss


### Optimizer
We define the model optimization specific operations. We use two optimizers here; Adam and SGD. Using Adam only causes the model to exhibit some undesired behaviors in the long run. Therefore, Adam used to get a good initial estimate for the SGD and use SGD from that point onwards.

In [17]:
print('Defining Optimizers')

with tf.device("/device:GPU:1"):
    
    # These are used to decay learning rate over time
    global_step = tf.Variable(0, trainable=False)
    inc_gstep = tf.assign(global_step,global_step + 1)

    # We use two optimizers, when the optimizer changes
    # we reset the global step
    reset_gstep = tf.assign(global_step,0)

    # Calculate decaying learning rate...no lower than 0.00001
    learning_rate = tf.maximum(tf.train.exponential_decay(0.005, global_step, decay_steps=1, decay_rate=0.95,\
                                                          staircase=True), 0.00001)

    sgd_learning_rate = tf.maximum(tf.train.exponential_decay(0.005, global_step, decay_steps=1, decay_rate=0.95,\
                                                              staircase=True), 0.00001)

    # We use two optimizers: Adam and naive SGD
    # using Adam in the long run produced undesirable results 
    # (e.g.) sudden fluctuations in BLEU
    # Therefore we use Adam to get a good starting point for optimizing
    # and then switch to SGD from that point onwards
    with tf.variable_scope('Adam'):
        optimizer = tf.train.AdamOptimizer(learning_rate)
    with tf.variable_scope('SGD'):
        sgd_optimizer = tf.train.GradientDescentOptimizer(sgd_learning_rate)

    # Calculates gradients with clipping for Adam
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
    optimize = optimizer.apply_gradients(zip(gradients, v))

    # Calculates gradients with clipping for SGD
    sgd_gradients, v = zip(*sgd_optimizer.compute_gradients(loss))
    sgd_gradients, _ = tf.clip_by_global_norm(sgd_gradients, 5.0)
    sgd_optimize = optimizer.apply_gradients(zip(sgd_gradients, v))

    # Make sure gradients exist flowing from decoder to encoder
    for (g_i,v_i) in zip(gradients,v):
        assert g_i is not None, 'Gradient none for %s'%(v_i.name)
    
print('\tDone')

Defining Optimizers
	Done


### Resetting Train and Test States
We here define the state resetting functions

In [18]:
with tf.device("/device:GPU:1"):
    
    # Reset training state
    reset_train_state = tf.group(tf.assign(saved_output, tf.zeros([batch_size, num_nodes])),
                                 tf.assign(saved_state, tf.zeros([batch_size, num_nodes]))
                                )

    reset_test_state = tf.group(
        saved_test_output.assign(tf.zeros([batch_size, num_nodes])),
        saved_test_state.assign(tf.zeros([batch_size, num_nodes]))
                                 )


 ## Running the Neural Machine Translator
 
 With all the relevant TensorFlow operations defined we move on to defining several functions related to executing our NMT model as well as runnning the model to obtain translations for previously unseen source sentences.

### Functions for Evaluating and Printing Results

Next we define two functions to print and save the prediction results for training data as well as testing data, and finally define a function to obtain candidate and reference data to calculate the BLEU score.

In [19]:
def print_and_save_train_predictions(du_labels, tr_pred, rand_idx, train_prediction_text_fname):
    '''
    Use this to print some predicted training samples and save it to file
    du_labels: Decoder's unrolled labels (this is a list of dec_num_unrollings 
    where each item is [batch_size, vocabulary_size])
    tr_pred: This is an array [dec_num_unrollings*batch_size, vocabulary_size] array
    rand_idx: Some random index we use to pick a data point to print
    train_prediction_text_fname: The file we save the prediction results into
    '''

    # This print_str will be written to the text file as well as printed here
    print_str = 'Actual: ' 
    
    # We can get each label corresponding to some sentence by traversing the
    # concatenated labels array ([dec_num_unrollings*batch_size, vocabulary_size])
    # with a batch_size stride
    for w in np.argmax(np.concatenate(du_labels,axis=0)[rand_idx::batch_size],axis=1).tolist():
        # Update the print_str
        print_str += tgt_reverse_dictionary[w] + ' '
        # When we encounter the end of sentence </s> we stop printing
        if tgt_reverse_dictionary[w] == '</s>':
            break
    print(print_str)
    
    # Write to file
    with open(os.path.join(log_dir, train_prediction_text_fname),'a',encoding='utf-8') as fa:                
        fa.write(print_str+'\n')  

    # Now print the predicted data by following the same procedure as above
    print()
    print_str = 'Predicted: '
    for w in np.argmax(tr_pred[rand_idx::batch_size],axis=1).tolist():
        print_str += tgt_reverse_dictionary[w] + ' '
        # When we encounter the end of sentence </s> we stop printing
        if tgt_reverse_dictionary[w] == '</s>':
            break
    print(print_str)
    with open(os.path.join(log_dir, train_prediction_text_fname),'a',encoding='utf-8') as fa:                
        fa.write(print_str+'\n')    
    
    
def print_and_save_test_predictions(test_du_labels, test_pred_unrolled, batch_id, test_rand_idx, test_prediction_text_fname):
    '''
    Use this to print some predicted training samples and save it to file
    test_du_labels: Decoder's unrolled labels (this is a list of dec_num_unrollings 
    where each item is [batch_size, vocabulary_size])
    test_pred_unrolled: This is an array [dec_num_unrollings*batch_size, vocabulary_size] array
    batch_id: We need this to retrieve the actual sentence for the predicted 
    test_rand_idx: Some random index we use to pick a data point to print
    test_prediction_text_fname: The file we save the prediction results into
    '''
    
    # Print the actual sentence
    print('DE: ',test_source_sent[(batch_id*batch_size)+test_rand_idx])
    # print_str is the string we display as results and write to a file
    print_str = '\t EN (TRUE):' + test_target_sent[(batch_id*batch_size)+test_rand_idx]
    print(print_str + '\n')

    # Printing predictions
    print_str = '\t EN (Predicted): ' 
    
    for test_pred in test_pred_unrolled:                            
        print_str += tgt_reverse_dictionary[test_pred[test_rand_idx]] + ' '
        if tgt_reverse_dictionary[test_pred[test_rand_idx]] == '</s>':
            break
    print(print_str + '\n')

    # Write the results to text file
    with open(os.path.join(log_dir, test_prediction_text_fname),'a',encoding='utf-8') as fa:                                
        fa.write(print_str+'\n') 
        
def create_bleu_ref_candidate_lists(all_preds, all_labels):
    '''
    Creates two lists (candidate list and reference list) for calcluating BLEU
    all_preds: All the predictions
    all_labels: Correspondign all the actual labels
    Returns
    cand_list: List (sentences) of lists (words in a sentence)
    ref_list: List (sentences) of lists (words in a sentence)
    '''
    bleu_labels, bleu_preds = [],[]
    
    # calculate bleu score:        
    # We iterate batch_size times as i=0,1,2,...,batch_size while grabbing 
    # i, i+batch_size, i+2*batch_size, i+3*batch_size elements from all_labels and all_preds
    # This because the labels/predicitons belonging to same sentence are interleaved by batch_size 
    # due to the way concatenate labels and predictions
    # Taking elements interleaved by batch_size gives the sequence of words belonging to the same sentence
    ref_list, cand_list = [],[]
    for b_i in range(batch_size):
        tmp_lbl = all_labels[b_i::batch_size]            
        tmp_lbl = tmp_lbl[np.where(tmp_lbl != tgt_dictionary['</s>'])]            
        ref_str = ' '.join([tgt_reverse_dictionary[lbl] for lbl in tmp_lbl])
        ref_list.append([ref_str])

        tmp_pred = all_preds[b_i::batch_size]
        tmp_pred = tmp_pred[np.where(tmp_pred != tgt_dictionary['</s>'])]
        cand_str = ' '.join([tgt_reverse_dictionary[pre] for pre in tmp_pred])
        cand_list.append(cand_str)

    return cand_list, ref_list

### Defining a Single Step of Training

We now define a function to train the NMT model for a single step. It takes in encoder inputs, decoder inputs and decoder labels and train the NMT for a single step.

In [20]:
def train_single_step(eu_data, du_data, du_labels):
    with tf.device("/device:GPU:1"):
        '''
        Define a single training step
        eu_data: Unrolled encoder inputs (word embeddings)
        du_data: Unrolled decoder inputs (word embeddings)
        du_labels: Unrolled decoder outputs (one hot encoded words)
        '''
        # Fill the feed dict (Encoder)
        feed_dict = {}
        for ui,dat in enumerate(eu_data):            
            feed_dict[enc_train_inputs[ui]] = dat    


        # Fill the feed dict (Decoder) 
        for ui,(dat,lbl) in enumerate(zip(du_data,du_labels)):            
            feed_dict[dec_train_inputs[ui]] = dat
            feed_dict[dec_train_labels[ui]] = lbl
            # The mask the </s> items from being part of the loss by 
            # setting mask position to zero (0) and non-mask positions to one (1)
            d_msk = (np.logical_not(np.argmax(lbl,axis=1)==tgt_dictionary['</s>'])).astype(np.int32).reshape(-1,1)
            feed_dict[dec_train_masks[ui]] = d_msk

        # ======================= OPTIMIZATION ==========================
        # Using Adam in long term gives very weird behaviors in loss
        # so after 20000 iterations we change the optimizer to SGD
        if (step+1)<20000:
            _,l,tr_pred = sess.run([optimize,loss,train_prediction], feed_dict=feed_dict)
        else:
            _,l,tr_pred = sess.run([sgd_optimize,loss,train_prediction], feed_dict=feed_dict)
        
    return l, tr_pred

### Defining Data Generators and Other Related Variables

Here we load the word embeddings and some other things as well as define a function to retrieve data generators

In [21]:
# This is where all the results will be logged into
log_dir = 'logs'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

# Filenames of the logs
train_prediction_text_fname = 'train_predictions.txt'
test_prediction_text_fname = 'test_predictions.txt'

with tf.device("/device:GPU:1"):
    # Some configuration for the TensorFlow session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement=True
    sess = tf.InteractiveSession(config=config)

    # Initialize variables
    tf.global_variables_initializer().run()

    # Load the word embeddings
    src_word_embeddings = src_emb_mat
    tgt_word_embeddings = tgt_emb_mat

    # Defining data generators
    def define_data_generators(batch_size, enc_num_unrollings, dec_num_unrollings):

        # Training data generators (Encoder and Decoder)
        enc_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=enc_num_unrollings,is_source=True, is_train=True)
        dec_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=dec_num_unrollings,is_source=False, is_train=True)

        # Testing data generators (Encoder and Decoder)
        test_enc_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=enc_num_unrollings,is_source=True, is_train=False)
        test_dec_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=dec_num_unrollings,is_source=False, is_train=False)

        return enc_data_generator,dec_data_generator,test_enc_data_generator,test_dec_data_generator


### Running Training and Testing for NMT

With all the TensorFlow operations, helper functions defined we train and test the NMT system.

In [22]:
with tf.device("/device:GPU:1"):
    
    
    # Training and test BLEU scores
    train_bleu_scores_over_time,test_bleu_scores_over_time = [],[]
    # Loss over time
    loss_over_time = []

    # Labels and predictions required to calculate the BLEU scores
    # For both train and test data
    train_bleu_refs, train_bleu_cands = [],[]
    test_bleu_refs, test_bleu_cands = [],[]

    # Number of steps the training goes for
    num_steps = 100001
    avg_loss = 0

    # Defining data generators for encoder/decoder and training/testing
    enc_data_generator, dec_data_generator, \
    test_enc_data_generator, test_dec_data_generator = \
    define_data_generators(batch_size, enc_num_unrollings, dec_num_unrollings)

    print('Started Training')
    
    start_sec = time()

    for step in range(num_steps):

        # input (encoder) unrolling length: 40
        # output (decoder) unrolling length: 60
        if (step+1)%100==0:
            print('.',end='')

        # Sample a random batch of IDs from training data
        sent_ids = np.random.randint(low=0,high=train_inputs.shape[0],size=(batch_size))

        # Getting an unrolled set of data batches for the encoder
        eu_data, _, _ = enc_data_generator.unroll_batches(sent_ids=sent_ids)

        # Getting an unrolled set of data batches for the decoder
        du_data, du_labels, _ = dec_data_generator.unroll_batches(sent_ids=sent_ids)

        # Train for single step
        l, tr_pred = train_single_step(eu_data, du_data, du_labels)

        # We don't calculate BLEU scores all the time as this is expensive, 
        # it slows down the code
        if np.random.random()<0.1:

            # all_labels are labels obtained by concatinating all the labels in batches
            all_labels = np.argmax(np.concatenate(du_labels,axis=0),axis=1)
            # all_preds are predictions for all unrolled steps
            all_preds = np.argmax(tr_pred,axis=1)

            # Get training BLEU candidates and references
            batch_cands, batch_refs = create_bleu_ref_candidate_lists(all_preds, all_labels)

            # Accumulate training candidates/references for calculating
            # BLEU later
            train_bleu_refs.extend(batch_refs)
            train_bleu_cands.extend(batch_cands)

        if (step+1)%500==0:  

            # Writing actual and predicte data to train_prediction.txt file for some random sentence
            print('Step ',step+1)
            with open(os.path.join(log_dir, train_prediction_text_fname),'a') as fa:                                
                fa.write('============= Step ' +  str(step+1) + ' =============\n') 

            rand_idx = np.random.randint(low=1,high=batch_size)
            print_and_save_train_predictions(du_labels, tr_pred, rand_idx, train_prediction_text_fname)        

            # Calculating the BLEU score for the accumulated candidates/references
            bscore = 0.0
            bscore = corpus_bleu(train_bleu_refs,train_bleu_cands,smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4)
            train_bleu_scores_over_time.append(bscore)
            print('(Train) BLEU (%d elements): '%(len(train_bleu_refs)),bscore)

            # Reset the candidate/reference accumulators
            train_bleu_refs, train_bleu_cands = [],[]

            # Write BLEU score to file
            with open(log_dir + os.sep +'blue_scores.txt','a') as fa_bleu:
                fa_bleu.write(str(step+1) +','+str(bscore)+'\n')

            with open(os.path.join(log_dir, train_prediction_text_fname),'a') as fa:                
                fa.write('(Train) BLEU: %.5f\n'%bscore)        

        avg_loss += l  # Update average loss

        sess.run(reset_train_state) # Resetting hidden state for each batch

        # ============================= TEST PHASE ==================================
        if (step+1)%1000==0:
            # Calculate average loss
            print('============= Step ', str(step+1), ' =============')
            print('\t Loss: ',avg_loss/1000.0)
            loss_over_time.append(avg_loss/1000.0)

            # Write losses to file
            with open(log_dir + os.sep + 'losses.txt','a') as fa_loss:
                fa_loss.write(str(step+1) +','+str(avg_loss/1000.0)+'\n')

            with open(os.path.join(log_dir, train_prediction_text_fname),'a') as fa:                                
                fa.write('============= Step ' +  str(step+1) + ' =============\n') 
                fa.write('\t Loss: %.5f\n'%(avg_loss/1000.0))

            avg_loss = 0.0

            # Increase gstep to decay learning rate
            sess.run(inc_gstep)

            # reset global step when we change the optimizer
            if (step+1)==20000: 
                sess.run(reset_gstep)

            print('=====================================================')
            print('(Test) Translating test sentences ...')        

            print('Processing test data ... ')

            # ===================================================================================
            # Predictions for Test data
            for in_i in range(test_inputs.shape[0]//batch_size):

                # Generate encoder / decoder data for testing data
                test_eu_data, test_eu_labels, _ = test_enc_data_generator.unroll_batches(sent_ids=np.arange(in_i*batch_size,(in_i+1)*batch_size))
                test_du_data, test_du_labels, _ = test_dec_data_generator.unroll_batches(sent_ids=np.arange(in_i*batch_size,(in_i+1)*batch_size))

                # fill the feed dict
                feed_dict = {}
                for ui,(dat,lbl) in enumerate(zip(test_eu_data,test_eu_labels)):            
                    feed_dict[enc_test_input[ui]] = dat             

                # Get predictions out with decoder          
                # run prediction calculation this returns a list of prediction dec_num_unrollings long
                test_pred_unrolled = sess.run(test_predictions, feed_dict=feed_dict)

                # We print a randomly selected sample from each batch
                test_rand_idx = np.random.randint(0,batch_size) # used for printing test output

                print_and_save_test_predictions(test_du_labels, test_pred_unrolled, in_i, test_rand_idx, test_prediction_text_fname)

                # Things required to calculate test BLEU score
                all_labels = np.argmax(np.concatenate(test_du_labels,axis=0),axis=1)
                all_preds = np.concatenate(test_pred_unrolled, axis=0)
                batch_cands, batch_refs = create_bleu_ref_candidate_lists(all_preds, all_labels)
                test_bleu_refs.extend(batch_refs)
                test_bleu_cands.extend(batch_cands)

                # Reset the test state
                sess.run(reset_test_state)

            # Calculate test BLEU score
            test_bleu_score = 0.0
            test_bleu_score = corpus_bleu(test_bleu_refs,test_bleu_cands,
                                          smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4)
            test_bleu_scores_over_time.append(test_bleu_score)
            print('(Test) BLEU (%d elements): '%(len(test_bleu_refs)),test_bleu_score)

            test_bleu_refs, test_bleu_cands = [],[]        
            print('=====================================================')
            
    end_sec = time()
    elapse_mins = (end_sec - start_sec)/60.0
    elapse_hrs = elapse_mins/60.0
    print()
    print('Training Elapse Time is {:4f} hours'.format(elapse_hrs))

Started Training


c:\users\trevo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
c:\users\trevo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
c:\users\trevo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:85: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


.....Step  500
Actual: In this case   any user who connects from the local host with the correct password for the anonymous user will be allowed access   with the privileges associated with the anonymous ##AT##-##AT## user account    </s> 

Predicted: The the hotel    of ##AT##-##AT## are the the world of of the world of  the world of  be a to to   the same   the world  ##AT##-##AT##     </s> 
(Train) BLEU (450 elements):  0.11662043938616697
.....Step  1000
Actual: This is what the real story of the Asian economic miracle is all about   The Asian countries that have succeeded are those who took charge of their development agenda and used the ODA given to them to develop the capabilities that they really needed   not those that the donor countries were pushing on them   

Predicted: The page a you most estate of the most and and  a the the   new and  you a  a of have a of the own   the to of  the the  the the world of you are you to     you Lord of  not  the   
(Train) BLEU (400 elemen

DE:  Obwohl das Nazi ##AT##-##AT## Regime die Buddhistische Gemeinde in Berlin , die seit 1936 aktiv gewesen war , schloss und kurzzeitig deren Begrnder Martin Steinke 1941 inhaftierte , verfolgte es die Buddhisten nicht generell .

	 EN (TRUE):Although the Nazi regime closed the Buddhistische Gemeinde ( Buddhist Society ) in Berlin , which had been active from 1936 , and briefly arrested its founder Martin Steinke in 1941 , they generally did not persecute Buddhists .


	 EN (Predicted): In the first time of the first time of the first time of the first time of the first time of the first time of the first time    </s> 

DE:  Bamberg , die &quot; Traumstadt der Deutschen &quot; , seine aufgeschlossenen Menschen und seine romantische Umgebung wird auch Sie begeistern , denn sie bietet fr jeden etwas .

	 EN (TRUE):The beauty and rich cultural life of this town can be enjoyed at any time of year . Soak up the summer sun whilst relaxing at one of the many sidewalk cafs in the historic ol

DE:  Das Hotel Opera befindet sich in der Nhe des Royal Theatre , Kongens Nytorv , &apos; Stroget &apos; und Nyhavn .

	 EN (TRUE):Hotel Opera is situated near The Royal Theatre , Kongens Nytorv , &quot; Strget &quot; and fascinating Nyhavn .


	 EN (Predicted): The hotel is located in the heart of the city   the hotel is located in the heart of the city    </s> 

DE:  Das ist viel einfacher ... Nein , streiten Sie nicht mit mir ... es ist einfacher ... ach , wie auch immer !

	 EN (TRUE):This is far more easy ... no , don &apos;t argue with me ... it is easier ... ah whatever !


	 EN (Predicted): The hotel is a good location in the city    </s> 

DE:  Eine Woche spter wird Dianne Feinstein , Vorstandsvorsitzende der Inspektoren , als Nachfolgerin Moscones ernannt . Sie ist die erste Brgermeisterin der Stadt .

	 EN (TRUE):Returning by the parallel Stockton or Powell will give you a better feeling of the day to day life of the residents , and are both good for those looking for import

...

DE:  Tarbet Gast ist Haus im ersten Nationalpark von Schottland aufgestellt und hat eine gehobene Position hoch ber dem Dorf von Tarbet und geniet spektakulre sdliche Blicke Bucht Lomond hinunter und nach der westlichen Seite von Ben Lomond .

	 EN (TRUE):Tarbet Guest House is situated In Scotland  s first National Park and has an elevated position high above the village of Tarbet and enjoys spectacular southerly views down Loch Lomond and towards the western side of Ben Lomond .


	 EN (Predicted): Combining the best of the most beautiful landscapes of the world   the new concept of the new concept of the new concept of the new concept of the new concept of the world    </s> 

DE:  Dieses 4 ##AT##-##AT## Sterne ##AT##-##AT## Landhotel aus dem 18. Jahrhundert inmitten einer lndlichen Umgebung ist nur eine kurze Fahrt vom Stadtzentrum von Londonderry und vom rtlichen Flughafen entfernt .

	 EN (TRUE):This 4 ##AT##-##AT## star 18th century country house hotel is situated in the countrysi

DE:  Auch ist , so denkt Dr. Gutherz , bereits die erste Seite sehr viel versprechend , da sie eine Definition des klinischen Psychotrauma ##AT##-##AT## Begriffes enthlt , der er gnzlich zustimmen kann .

	 EN (TRUE):At the rhetorical climax of this summary , Dr Goodheart comes across some sentences expressed with great pathos .


	 EN (Predicted): I think that I have to be able to see the new version of the new version of the new version of the new version of the new version    </s> 

DE:  Das ist eine Metapher , wird jemand von der Propagandaabteilung entgegnen .

	 EN (TRUE):It &apos;s only a metaphor , people from the propaganda department will say .


	 EN (Predicted): The new camera is a new      </s> 

DE:  Es handelt sich um ein ziemlich einfaches Protokoll ; TFTP macht aber manchmal Probleme .

	 EN (TRUE):This is a fairly simple protocol , but sometimes there are problems trying to get it to work .


	 EN (Predicted): I have a lot of the same    </s> 

DE:  in dieser Option e

DE:  Unsere krzlich renovierten Ferienwohnungen zur Selbstversorgung in Obertraun sind weniger als eine Gehminute vom Ufer des Hallsttter Sees entfernt .

	 EN (TRUE):Enjoy a warm and friendly welcome at the Obertrauner Hof , located at the heart of the tranquil village of Obertraun in the Salzkammergut , a delightful 10 ##AT##-##AT## minute walk from the lakeside .


	 EN (Predicted): Verily thee   thy loins of thy servants thy brother thy servants thy brother    </s> 

DE:  15. einem Dritten bei dem Verstoss gegen eine dieser Regeln zu helfen .

	 EN (TRUE):15. assist any third party in engaging in any activity prohibited by these Terms .


	 EN (Predicted): percent of the new version of the new version of the new version of the new version of the new version    </s> 

DE:  Es war staubig , das Bad schmutzig . Sogar die Beleuchtung an der Wand im Flur ( Seitengebude ) war richtig verstaubt .

	 EN (TRUE):It was rather old fashioned in the decoration .


	 EN (Predicted): I think that

.....Step  97500
Actual: Room Notes : DOUBLE SUPERIOR ROOM ; Our superior double rooms have recently to the same high standard as our other rooms   but are larger with a lounge are and comfortable sofa    </s> 

Predicted: Room Notes : Double ROOM - FOR for rooms rooms rooms are a been offer highest double standard and well rooms rooms    the available than a total  a a     </s> 
(Train) BLEU (540 elements):  0.3317723425851313
.....Step  98000
Actual: The English newsletter is sent by eMail every 3 ##AT##-##AT## 4 months    </s> 

Predicted: It new is is a to the by year ##AT##-##AT## star     </s> 
(Train) BLEU (510 elements):  0.30561115103138864
============= Step  98000  =============
	 Loss:  0.852690947920084
(Test) Translating test sentences ...
Processing test data ... 
DE:  &#124; Ferienwohnungen 1 Zi &#124; Ferienhuser &#124; Landhuser &#124; Autovermietung &#124; Last Minute Angebote ! !

	 EN (TRUE):&#124; 1 Bedroom Apts &#124; Holiday houses &#124; Rural Homes &#124; Car 

DE:  Zerstren Sie alle Blcke , um zur nchsten Ebene zu gelangen .

	 EN (TRUE):Destroy all the blocks to advance to the next level .


	 EN (Predicted): a new version of the new version of the new version of the new version    </s> 

DE:  Das ist eine Metapher , wird jemand von der Propagandaabteilung entgegnen .

	 EN (TRUE):It &apos;s only a metaphor , people from the propaganda department will say .


	 EN (Predicted): The new version of the new version of the new version of the new version of the new version of the new version of the new version of the new version of the new version of the new version of the new version of the new version    </s> 

DE:  Jeder Wikitraveler kann Artikel verndern , neue Seiten erstellen und sogar Informationen ber die Seite selbst berschreiben .

	 EN (TRUE):Any Wikitraveller can change articles , rewrite navigation areas , even overwrite information about the site itself .


	 EN (Predicted): Poker your mobile phone is a new version of the latest ver